- Goal
    - Create synthetic data using IDM
    - Learn parameters of that data using particle filtering
    - Aliter 7 Feb: Learn using CEM idea, fitness function and then distb and then sample
- Learning
    - Need at least 2 vehicles so that there is a neighbor in the front
    - Otherwise src/1d/driver/lane_follower_driver errors
    - That is why the `AutomotiveDrivingModels/doc/1DMobius` stuff does 
    not work with 1 car only (needs at least 2 cars)
    - The tutorial does not work with `gen_straight_roadway` because that
    generates a roadway of type AutomotiveDrivingModels.roadway as opposed to
    AutomotiveDrivingModels.StraightRoadway
- Open question
    - IDM won't work unless there is a car in front (errors saynig nothing in sight)
    - We are focusing on learning the params of the second car here. Is that sound sensible?
- Flow of code
    - Call the required `usings`
    - Define functions required
    - Actual running things
        - Generate true trajectory
        - Generate a set of particles uniformly between sensible range of values
        - Compute fitness, sort and select numtop
        - Fit a distribution over this
        - Resample particles
- Scenario
    - Slower car in front (car 1)
    - Faster car behind (car 2)
    - We want to estimate params of car 2
- Feb 10: Adding timegap_des as our 2nd param in the 2 car, 1D, IDM scenario
    - Make code capable of handling 2d param. So far, had only scalar param
    i.e v_des
    - Fitting 2D distributions is required now
- Feb 22
    - the `rec` generated using `simulate` is a devious monster
    - It stores the last timestep in the 1st entry and first timestep in last entry
    - Be careful
- Mar 27
    - Inspired by behavior test written by Maxime to convert this code to work with general scene
    instead of scene1D and state1D as previously
    - Scaling up: scenario is to have 3 different speed vehicles starting in adjacent lanes. Each
    car will have an associated bucket of particles and at the end we whould be able to uncover
    true params for all three cars
    - Disclaimer: This will definitely work since we have confirmed that we can uncover params for one 
    car when it is the leader (i.e. unaffected by any car in front). The current exercise is to scale up the
    code to work with multiple vehicles
    - Idea is that every vehicles associated bucket of particles will evolve differently as it sees more
    of the true trajectory. This is in contrast to original idea where we thought we will have all the buckets
    pooled together to inform the final distribution

In [1]:
using AutomotiveDrivingModels
using AutoViz
using Reel
using Interact
using StatsBase # For random particle generation
using Test
using Distributions
using Pkg

┌ Info: Recompiling stale cache file /home/raunak/.julia/compiled/v1.1/AutomotiveDrivingModels/ZEfHM.ji for AutomotiveDrivingModels [99497e54-f3d6-53d3-a3a9-fa9315a7f1ba]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/raunak/.julia/compiled/v1.1/AutoViz/w0rHu.ji for AutoViz [82aa6e0c-a491-5edf-8d4b-c16b98e4ea17]
└ @ Base loading.jl:1184


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/ac92513fc97126f4d4502d0637c43de8cdb5fca9-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/ac92513fc97126f4d4502d0637c43de8cdb5fca9-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [112]:
"""
Helper function for tests: init_scene: generate a scene with 3 lane roadway and cars at 10,20,30 m in lane1,2,3
"""
function init_scene(road_length)
    roadway = gen_straight_roadway(3,road_length)

    veh_state = VehicleState(Frenet(roadway[LaneTag(1,1)], 10.0), roadway, 0.)
    veh1 = Vehicle(veh_state, VehicleDef(), 1)
    veh_state = VehicleState(Frenet(roadway[LaneTag(1,2)], 20.0), roadway, 0.)
    veh2 = Vehicle(veh_state, VehicleDef(), 2)
    veh_state = VehicleState(Frenet(roadway[LaneTag(1,3)], 30.0), roadway, 0.)
    veh3 = Vehicle(veh_state, VehicleDef(), 3)

    scene = Scene()
    push!(scene, veh1)
    push!(scene, veh2)
    push!(scene, veh3)
end

init_scene

In [5]:
"""
gen_traj: Generate trajectory using a given particle for default 100 timestep trajectory

-----------Arguments:
`particle` Dict with keys as parameter names of IDM and values as parameter values
`nticks` Duration of simulation

-----------Other functions called:`init_scene`
"""
function gen_traj(particle;nticks=100,timestep=0.1)
    scene = init_scene()
    models = Dict{Int, LaneFollowingDriver}()
    models[1] = IntelligentDriverModel(;particle...)
    models[2] = IntelligentDriverModel(v_des=12.0)

    # Simulate for nticks (default 100) time steps
    timestep = 0.1
    rec = QueueRecord(Vehicle1D, nticks+1, timestep)
    simulate!(LaneFollowingAccel, rec, scene, roadway, models, nticks)

    # Extract the position and velocity of nticks timesteps
    # X stores this as 2d array. Timestep is the row, col 1 is pos,vel is pos2
    n_cars = scene.n
    n_ticks = nticks
    X = Array{Float64}(undef, n_ticks, 1)

    for t in 1:n_ticks
        f = rec.frames[n_ticks - t + 1]
        
        # BAD: 2 cars in scene therefore loop has only 1 elem
        for c in 1:1 #Was 2:sc
            s = f.entities[c].state
            X[t, 1] = s.s #position
        end
    end
    return X, rec
end

gen_traj

In [152]:
"""
Hallucinate a step forward given a specific car
---------Arguments:
`scene` Scene to start hallucination from
`particle` Dict with key as IDM parameter name and value as param val
`car_id` Identity of the car of interest

-------Returns: Hallucinated position of car of interest

---------NOTES:------------
- For now, we hallucinate the car of interest with the particle. Other cars are assumed to be
driving with IDM(v_des = 10). They don't matter in the particle update of the car of interest
(FOR NOW)
- We return only the position of the car along the lane. We don't even return which lane it is
in information. In the future, we want to return 2D position and measure 2D likelihood value somehow
"""
function hallucinate_a_step(scene,particle;car_id=-1)
    if car_id==-1 @show "Please give valid car_id" end
    n_cars = scene.n 
    models = Dict{Int, DriverModel}()
    
    # Create driver models for all the cars in the scene
    for i in 1:n_cars
        if i == car_id
            models[i] = IntelligentDriverModel(;particle...)
        else
            # TODO: RESEARCH QUESTION: What drives the other vehicles in the hallucination
            models[i] = IntelligentDriverModel(v_des=10.0)
        end
    end
    
    n_steps = 1
    dt = 0.1
    rec = SceneRecord(n_steps, dt)
    simulate!(rec, scene, roadway, models, n_steps)
    
    X = Array{Float64}(undef,n_steps, 1)

    for t in 1:n_steps
        f = rec.frames[n_steps - t + 1]
        
        for c in car_id:car_id
            s = f.entities[c].state.posF
            X[t, 1] = s.s #position
        end
    end
    return X[1]
end

hallucinate_a_step

In [166]:
"""
to_matrix_form: Return information and more workable form for initial particle set

---------Arguments:
`p_set_dict` Dictionary with parameters of IDM as keys and associated value as array of particles

---------Returned things that need explanation:
`p_mat` Matrix with each row corresponding to a different parameter of IDM, and each column to a diff particle
`params` Array of parameters eg: [:v_des,:σ]
`vec_val_vec` Array with each element being another array. This array contains the values i.e diff particles
"""
function to_matrix_form(p_set_dict)
    # Get the number of particles
    # Extract keys (i.e. params) and corresponding array of values
    num_p = -100
    num_params = length(keys(p_set_dict))
    
    params = Array{Symbol}(undef,num_params,1)
    vec_val_vec = Array{Array}(undef,num_params,1) #Array containing associated values for each key
    for (kk,kv) in enumerate(p_set_dict)
        num_p = length(kv[2])
        params[kk] = kv[1]
        vec_val_vec[kk] = kv[2]
    end
    
    # Create a matrix with different rows being different parameters and diff cols being diff particles
    p_mat = hcat(vec_val_vec...)'
    
    return p_mat, params, vec_val_vec
end

to_matrix_form

In [168]:
"""
compute_particle_likelihoods: Loop over the particles and score each of them

-----Other functions called: `to_matrix_form`,`hallucinate_a_step`
"""
function compute_particle_likelihoods(f,trupos,p_set_dict;car_id=-1)
    if car_id==-1 @show "Please give valid car_id" end
    timestep = 0.1 #TODO: Remove hardcoding
    p_mat, params, vec_val_vec = to_matrix_form(p_set_dict)
    
    num_params=size(p_mat)[1]
    num_p = size(p_mat)[2]
    lkhd_vec = Array{Float64}(undef,num_p)
    for i in 1:num_p    
        # Create dict version for a single particle
        p_dict = Dict()
        for j in 1:num_params
            p_dict[params[j]]=vec_val_vec[j][i]
        end
        
        std_dev_acc = p_dict[:σ]
        
        # hack to avoid the std_dev_pos become negative and error Normal distb
        if std_dev_acc <= 0 std_dev_acc = 0.1 end
        
        # TODO: This math needs to be verified from random variable calculations
        std_dev_pos = timestep*timestep*std_dev_acc
            
        hpos = hallucinate_a_step(f,p_dict,car_id=car_id)
        
        lkhd_vec[i] = pdf(Normal(hpos,std_dev_pos),trupos[1])
    end
    return lkhd_vec,p_mat,params
end

compute_particle_likelihoods

In [ ]:
"""
update_p_one_step: Update particles given one step of true data

------------Arguments that need explanation:
`p_set_dict` Dictionary with parameters of IDM as keys and associated value as array of particles
`f` Frame to start hallucination from
`trupos` Resulting true position starting from frame f
`approach` Select "pf" or "cem"
`elite_fraction_percent` Required for the cem method to fit a distribution

------------Other functions called:`to_matrix_form`,`hallucinate_a_step`

------------Data structures that need explanation:
vec_val_vec: Vector of value_vectors: Array with each element being array of values

------------Returns:
`new_p_set_dict` Dictionary with keys as IDM parameters and values as array of particles

-----------NOTES
- This function updates associated particles over 1 step for one car
- I think frame and scene can be used as the same thing. Maybe techincally scene is an array
with each element in that array being a frame.
- This function will be called by a function that loops over all the cars present in a scene
"""
function update_p_one_step(f,trupos,p_set_dict;car_id=-1,approach="pf",elite_fraction_percent=20)
    if car_id==-1 @show "Provide valid carid" end
    
    lkhd_vec,p_mat,params = compute_particle_likelihoods(scene,trupos,p_set_dict,car_id=2)
    
    num_params = size(p_mat)[1]
    num_p = size(p_mat)[2]
    
    if approach=="pf"
        p_weight_vec = weights(lkhd_vec./sum(lkhd_vec)) # Convert to weights form to use julia sampling
        idx = sample(1:num_p,p_weight_vec,num_p)
        new_p_mat = p_mat[:,idx] #Careful that idx is (size,1) and not (size,2)
    end
    
    if approach=="cem"
        sortedidx = sortperm(lkhd_vec,rev=true)
        numtop = convert(Int64,ceil(num_p*elite_fraction_percent/100.0))
        best_particles = p_mat[:,sortedidx[1:numtop]] # elite selection
#         @show best_particles
        p_distribution = fit(MvNormal,best_particles) # fit distb using elites
        new_p_mat = rand(p_distribution,num_p) # sample num_p new particles from dist
    end
    
#     @show new_p_set_dict
    return new_p_set_dict
    
end

In [176]:
"""
Create a new dictionary with param and associated particle value array
"""
function to_dict_form(params,new_p_mat)
    num_params = length(params)
    new_p_set_dict = Dict()
    for k in 1:num_params
        new_p_set_dict[params[k]] = new_p_mat[k,:]
    end
    return new_p_set_dict
end

to_dict_form

In [177]:
params = [:v_des,:σ]
new_p_mat = [17.0 26.0 24.0 19.0 29.0; 0.1 0.1 0.7 0.4 0.1]
new_p_set_dict = to_dict_form(params,new_p_mat)

MethodError: MethodError: no method matching (::Colon)(::Int64, ::Array{Symbol,1})
Closest candidates are:
  Colon(::T<:Real, ::Any, !Matched::T<:Real) where T<:Real at range.jl:40
  Colon(::A<:Real, ::Any, !Matched::C<:Real) where {A<:Real, C<:Real} at range.jl:10
  Colon(::T, ::Any, !Matched::T) where T at range.jl:39
  ...

In [ ]:
"""
Run the CEM_PF and PF approach over an entire trajectory
------------------------------------------
Generate a ground truth trajectory of 100 timesteps.
Make your particle filter go step by step over this ground truth.
Hoepfully by the end of the trajectory, it has figured out the true parameters
"""
num_p = 100
# start:step:end and number of particles are the inputs to sample
v_particles = sample(10.0:1.0:30.0,num_p)
sig_particles = sample(0.1:0.1:1.0,num_p)
old_p_set_dict = Dict(:v_des=>v_particles,:σ=>sig_particles)

# Generate ground truth trajectory
pos_ground_truth,rec_ground_truth = gen_traj(Dict(:v_des=>25.0,:σ=>0.4))

# loop over the trajectory step by step
f_end_num = length(rec_ground_truth.frames)
    
for t in 1:f_end_num-1
#     @show t
    f = rec_ground_truth.frames[f_end_num - t + 1]
    trupos = pos_ground_truth[t]
    
    # Select approach either "cem" or "pf"
    new_p_set_dict = update_p_one_step(old_p_set_dict,f,trupos,approach="pf")
    old_p_set_dict = new_p_set_dict
end
    
#@show fit(MvNormal,old_p_mat) # Don't work because all elements identical
@show old_p_set_dict

# Visualize

In [ ]:
# Function: Return rec corresponding to generated traj
    # Will help visualizatoin
    # Calls init_scene
# Might be useful later
    # models[2] = IntelligentDriverModel(v_des=particle[1],s_min=particle[2],T=particle[3])
function gen_rec4vid(particle;nticks=100,timestep=0.1)
    scene = init_scene()
    models = Dict{Int, LaneFollowingDriver}()
    models[1] = IntelligentDriverModel(v_des=particle[1],σ = particle[2])
    models[2] = IntelligentDriverModel(v_des=12.0)

    # Simulate for nticks (default 100) time steps
    timestep = 0.1
    rec = QueueRecord(Vehicle1D, nticks+1, timestep)
    simulate!(LaneFollowingAccel, rec, scene, roadway, models, nticks)

    return rec
end

In [ ]:
# overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true,
#         pos=VecE2(veh.state.s-0.7, 3)) for veh in scene];
# render(scene, roadway, overlays, cam=cam, canvas_height=100)

cam = StaticCamera(VecE2(100.0,0.0), 4.75)
true_rec = gen_rec4vid([20.0 0.1],nticks=100)
rec = true_rec
@manipulate for frame_index in 1 : nframes(rec)
    render(rec[frame_index-nframes(rec)], roadway, cam=cam, canvas_height=100)
end

# LEARNING AND EXPERIMENTATION

In [ ]:
# LEARNING ABOUT MULTIDIM DISTB
# Test: Generate samples for a 2d distb
d2 = MvNormal(2,2.0) # first arg shows dimension, second shows std dev
qw = rand(d2,6) # Will generate 6 samples i.e. 6 columns

# Test: Fit 2d distribution
dx = Normal()
dy = Normal(2,1.0)
x = rand(dx,100)
y = rand(dy,100)

# Matrix with each column being a sample
# Total columns is total number of samples
# Total rows is number of parameters
# All entries in a row contain value from same param eg:v_des
data_matrix = vcat(x',y')
fit(MvNormal,data_matrix)

In [ ]:
using PyPlot

In [ ]:
num_samples = 2000
y1 = rand(Normal(10.0,5.0),num_samples)
y2 = rand(Normal(2.0,1.0),num_samples)
plot(1:num_samples,y1)
plot(1:num_samples,y2)

In [ ]:
using StatPlots

In [ ]:
StatPlots.plot(Normal(3,5),linewidth=4,size=(2500,2500))

In [ ]:
roadway = gen_straight_roadway(2,1000.0);

In [ ]:
scene = Scene1D()
push!(scene, Entity(State1D(10.0,  8.0), VehicleDef(), 1))
push!(scene, Entity(State1D(50.0, 12.5), VehicleDef(), 2))

cam = StaticCamera(VecE2(100.0,0.0), 4.75)
overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, pos=VecE2(veh.state.s-0.7, 3)) for veh in scene]
render(scene, roadway, overlays, cam=cam, canvas_height=100)

# Tests (serves as usage examples for functions)

In [79]:
# gen_traj
roadway = gen_straight_roadway(3,1000.0)
models = Dict{Int, DriverModel}()
models[1] = IntelligentDriverModel(v_des = 10.0)
models[2] = IntelligentDriverModel(v_des = 20.0)
models[3] = IntelligentDriverModel(v_des = 40.0)

# Initialize all vehicles at the same 0 point location on the road
veh_state = VehicleState(Frenet(roadway[LaneTag(1,1)], 10.0), roadway, 0.)
veh1 = Vehicle(veh_state, VehicleDef(), 1)
veh_state = VehicleState(Frenet(roadway[LaneTag(1,2)], 20.0), roadway, 0.)
veh2 = Vehicle(veh_state, VehicleDef(), 2)
veh_state = VehicleState(Frenet(roadway[LaneTag(1,3)], 30.0), roadway, 0.)
veh3 = Vehicle(veh_state, VehicleDef(), 3)

scene = Scene()
push!(scene, veh1)
push!(scene, veh2)
push!(scene, veh3)
# render(scene,roadway)

particle = Dict(:v_des=>25.0,:σ=>0.5)
X = hallucinate_a_step_scale(scene,2,particle)

# n_steps = 100
# dt = 0.1
# rec = SceneRecord(n_steps, dt)
# simulate!(rec, scene, roadway, models, n_steps)
# @manipulate for frame_index in 1 : nframes(rec)
#     render(rec[frame_index-nframes(rec)], roadway, canvas_height=100)
# end

20.015735112859463

In [62]:
@show rec.frames[10].entities[1].state.posF.ϕ
@show models[1].σ

((((rec.frames[10]).entities[1]).state).posF).ϕ = 0.0
(models[1]).σ = NaN


NaN

In [ ]:
# gen traj: Note that this requires a roadway to be defined because
# the simulate! method is called within gen_traj, and that needs a roadway defined
particle = Dict(:v_des=>25.0,:σ=>0.5)
next_pos = gen_traj(particle,nticks=1)
@show next_pos
@test isapprox(next_pos[1][1],51.2)

In [158]:
# Test: Initialize a scene
scene = init_scene(1000.0)
@test scene[1].state.posF.s==10.0
@test scene[2].state.posF.s==20.0
@test scene[3].state.posF.s==30.0

Test Passed

In [159]:
# Test: hallucinate a step
scene = init_scene(1000.0)
particle = Dict(:v_des=>25.0,:σ=>0.5)
@test isapprox(hallucinate_a_step(scene,particle,car_id=1),10.02,atol=0.1)
@test isapprox(hallucinate_a_step(scene,particle,car_id=2),20.02,atol=0.1)
@test isapprox(hallucinate_a_step(scene,particle,car_id=3),30.02,atol=0.15)

Test Passed

In [174]:
# Test: to_matrix_form
num_p = 5
# start:step:end and number of particles are the inputs to sample
v_particles = sample(10.0:1.0:30.0,num_p)
sig_particles = sample(0.1:0.1:1.0,num_p)
p_set_dict = Dict(:v_des=>v_particles,:σ=>sig_particles)
@show p_set_dict
p_mat, params, vec_val_vec = to_matrix_form(p_set_dict);
@show p_mat
@test params[1] == :v_des
@test params[2] == :σ
@test size(p_mat)[1] == 2
@test size(p_mat)[2] == 5
@test length(vec_val_vec) == 2
@test length(vec_val_vec[1]) == 5

p_set_dict = Dict(:v_des=>[17.0, 26.0, 24.0, 19.0, 29.0],:σ=>[0.1, 0.1, 0.7, 0.4, 0.1])
p_mat = [17.0 26.0 24.0 19.0 29.0; 0.1 0.1 0.7 0.4 0.1]


Test Passed

In [172]:
# Test: compute_particle_likelihoods
scene=init_scene(1000.0)

num_p = 5
v_particles = sample(10.0:1.0:30.0,num_p)
sig_particles = sample(0.1:0.1:1.0,num_p)
p_set_dict = Dict(:v_des=>v_particles,:σ=>sig_particles)

trupos = hallucinate_a_step(scene,Dict(:v_des=>25.0,:σ=>0.0),car_id=2)
lkhd_vec,p_mat,params = compute_particle_likelihoods(scene,trupos,p_set_dict,car_id=2)

@test length(lkhd_vec) == num_p
@test length(params) == 2
@test size(p_mat)[1] == 2
@test size(p_mat)[2] == 5

Test Passed

In [ ]:
# update_p_one_step
num_p = 5
# start:step:end and number of particles are the inputs to sample
v_particles = sample(10.0:1.0:30.0,num_p)
sig_particles = sample(0.1:0.1:1.0,num_p)
p_set_dict = Dict(:v_des=>v_particles,:σ=>sig_particles)
@show p_set_dict
# Generate ground truth trajectory
pos_ground_truth,rec_ground_truth = gen_traj(Dict(:v_des=>25.0,:σ=>0.2))
@show rec_ground_truth.frames[101].entities[1].state,pos_ground_truth[1]

update_p_one_step(p_set_dict,rec_ground_truth.frames[101],pos_ground_truth[1],
    approach="cem",elite_fraction_percent=60)

```
Expected output
p_set_dict = Dict(:v_des=>[11.0, 29.0, 27.0, 25.0, 30.0],:σ=>[1.0, 0.7, 0.4, 0.2, 0.7])
(rec_ground_truth.frames[101].entities[1].state, pos_ground_truth[1]) = 
(AutomotiveDrivingModels.State1D(50.0, 12.0), 51.2124675051445)
best_particles = [25.0 27.0 30.0; 0.2 0.4 0.7]
p_mat = [11.0 29.0 27.0 25.0 30.0; 1.0 0.7 0.4 0.2 0.7]
new_p_mat = [25.0191 27.8877 26.6066 26.3389 26.8579; 0.201913 0.488768 0.360664 0.333894 0.385794]
new_p_set_dict = Dict{Any,Any}(Pair{Any,Any}(:v_des, [25.0191, 27.8877, 26.6066, 26.3389, 26.8579]),
Pair{Any,Any}(:σ, [0.201913, 0.488768, 0.360664, 0.333894, 0.385794]))
```